## Text Mining 

Objectif : Fonction qui renvoie pour chaque numéro de cluster un mot qui décrit le mieux le cluster

Input : dataframe 
Output : cluster avec le title 

In [17]:
import pandas as pd
df = pd.read_parquet("flickr_data_clustered.parquet")

On commence par supprimer les stopwords

In [18]:
#fonction
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re


french_stopwords = {
    "le","la","les","de","des","du","un","une","et","à","au","aux",
    "en","pour","sur","avec","sans","ce","cette","ces","dans",
    "est","sont","été","être","avoir","Lyon","Demeure","Chaos" ,"France"
}

stopwords = ENGLISH_STOP_WORDS.union(french_stopwords)


def clean_stopwords(text) : 

    if not isinstance(text, str):
        return ""
    
    words = text.split()
    words = [w for w in words if w not in stopwords and len(w) > 2]
    return words
    

#apply the function to the 'title' column
df['cleaned_title_stopwords'] = df['title'].apply(clean_stopwords)

df['cleaned_tags_stopwords'] = df['tags'].apply(clean_stopwords)
   

In [19]:
df

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,...,date_upload_month,date_upload_year,taken_dt,upload_dt,url,tag_list,tag_count,cluster_kmeans,cleaned_title_stopwords,cleaned_tags_stopwords
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,...,2,2010.0,2010-02-28 15:11:00,2010-02-28 20:23:00,https://www.flickr.com/photos/30624617@N03/439...,"[chair, chaise, lyon, rhône, rhônealpes]",5,0,"[Chaises, vue]","[chair,lyon,rhône,chaise,rhônealpes]"
1,4394748717,35853470@N00,45.753270,4.862953,,,51.0,17,28,2,...,2,2010.0,2010-02-28 17:51:00,2010-02-28 17:52:00,https://www.flickr.com/photos/35853470@N00/439...,[],0,1,[],[]
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,...,2,2010.0,2010-02-28 17:29:00,2010-02-28 17:33:00,https://www.flickr.com/photos/11817998@N05/439...,"[365, iphone]",2,0,"[59/365, R46, V103, B163]","[365,iphone]"
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,...,2,2010.0,2010-01-28 20:15:00,2010-02-28 12:38:00,https://www.flickr.com/photos/11545749@N06/439...,"[avott, gift, nin, nineinchnails, screening, t...",6,1,"[2010-01-29, Toiou, Avott]","[nin,nineinchnails,gift,screening,toiou,avott]"
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,...,2,2010.0,2010-01-28 20:10:00,2010-02-28 12:38:00,https://www.flickr.com/photos/11545749@N06/439...,"[avott, gift, lyon, nin, nineinchnails, screen...",7,1,"[2010-01-28, Toiou, Avott]","[lyon,nin,nineinchnails,gift,screening,toiou,a..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168096,45746643811,146709606@N06,45.729498,4.951550,"car,autocar,interurbain,ligne,scolaire,temsa,l...",TEMSA LD 12 SB Plus - DGC,32.0,15,17,10,...,11,2018.0,2018-10-17 15:32:00,2018-11-06 11:40:00,https://www.flickr.com/photos/146709606@N06/45...,"[autocar, car, dcg, dietrichcarebusgroup, inte...",11,3,"[TEMSA, Plus, DGC]","[car,autocar,interurbain,ligne,scolaire,temsa,..."
168097,45746642111,146709606@N06,45.729498,4.951550,"car,autocar,interurbain,ligne,scolaire,temsa,l...",TEMSA LD 12 SB Plus - DGC,33.0,15,17,10,...,11,2018.0,2018-10-17 15:33:00,2018-11-06 11:40:00,https://www.flickr.com/photos/146709606@N06/45...,"[autocar, car, dcg, dietrichcarebusgroup, inte...",11,3,"[TEMSA, Plus, DGC]","[car,autocar,interurbain,ligne,scolaire,temsa,..."
168098,45680287992,146709606@N06,45.729498,4.951550,"car,autocar,tourisme,interurbain,van,hool,vanh...",VAN HOOL TDX27 Astromega - Van Hool,52.0,13,17,10,...,11,2018.0,2018-10-17 13:52:00,2018-11-05 13:00:00,https://www.flickr.com/photos/146709606@N06/45...,"[2018, 27, astromega, autocar, autocarexpo, ca...",16,3,"[VAN, HOOL, TDX27, Astromega, Van, Hool]","[car,autocar,tourisme,interurbain,van,hool,van..."
168099,44995017704,48633948@N08,45.771852,4.833115,"lyon,1714,4814we69,man,hess,nmt222,trolleybus,...",Lyon 1714 Rue Pouteau,20.0,14,18,10,...,11,2018.0,2017-10-18 14:20:00,2018-11-04 21:19:00,https://www.flickr.com/photos/48633948@N08/449...,"[1714, 4814we69, bus, buses, france, hess, lyo...",12,0,"[1714, Rue, Pouteau]","[lyon,1714,4814we69,man,hess,nmt222,trolleybus..."


Ensuite on on cherche le mot qui correspond à chaque cluster
df : le dataframe 
cluster_kmeans : le nom de la colonne qui contient le num de cluster auquel appartient la ligne
texte_cols : les colonnes où y a les textes qu'on va utiliser pour identifier les mots les plus fréquent 
top_k : nb de mots à garder = 1 puisque on cherche un seul mots par cluster

In [ ]:
from collections import Counter

def cluster_titles(
    df,
    cluster_kmeans="cluster_kmeans",
    text_cols=("cleaned_title_stopwords", "cleaned_tags_stopwords"),
    top_k=1
):
    """
    Retourne un titre (mot-clé) par cluster
    """
    cluster_labels = {}

    for cluster_id in sorted(df[cluster_kmeans].unique()):
        if cluster_id == -1:
            continue  # on ignore le bruit

        # sous-dataframe du cluster (seulement les ligne de ce cluster)
        dff = df[df[cluster_kmeans] == cluster_id]

        # concaténation des textes de toutes les lignes
        all_words = []
        #on parcours les deux colonnes de texte
        for col in text_cols:
            texts = dff[col].dropna()
            for t in texts:
                all_words.extend(t)
        # si la liste est vide donc pas de mots
        if not all_words:
            cluster_labels[cluster_kmeans] = "unknown"
            continue

        # comptage des mots
        counts = Counter(all_words)

        # mots les plus fréquents
        top_words = [w for w, _ in counts.most_common(top_k)]

        cluster_labels[cluster_id] = top_words

    return cluster_labels

df['cluster_name'] = df['cluster_kmeans'].map(
    cluster_titles(df, top_k=1)
)

In [21]:
df

,id,user,lat,long,tags,title,date_taken_minute,date_taken_hour,date_taken_day,date_taken_month,...,date_upload_year,taken_dt,upload_dt,url,tag_list,tag_count,cluster_kmeans,cleaned_title_stopwords,cleaned_tags_stopwords,cluster_name
0,4395181099,30624617@N03,45.754858,4.821710,"chair,lyon,rhône,chaise,rhônealpes",Chaises avec vue,11.0,15,28,2,...,2010.0,2010-02-28 15:11:00,2010-02-28 20:23:00,https://www.flickr.com/photos/30624617@N03/439...,"[chair, chaise, lyon, rhône, rhônealpes]",5,0,"[Chaises, vue]","[chair,lyon,rhône,chaise,rhônealpes]",[France]]
1,4394748717,35853470@N00,45.753270,4.862953,,,51.0,17,28,2,...,2010.0,2010-02-28 17:51:00,2010-02-28 17:52:00,https://www.flickr.com/photos/35853470@N00/439...,[],0,1,[],[],[Parc]
2,4394694699,11817998@N05,45.760655,4.846564,"365,iphone",59/365 - R46 V103 B163,29.0,17,28,2,...,2010.0,2010-02-28 17:29:00,2010-02-28 17:33:00,https://www.flickr.com/photos/11817998@N05/439...,"[365, iphone]",2,0,"[59/365, R46, V103, B163]","[365,iphone]",[France]]
3,4394803790,11545749@N06,45.784000,4.874072,"nin,nineinchnails,gift,screening,toiou,avott",2010-01-29 Toiou Avott Lyon,15.0,20,28,1,...,2010.0,2010-01-28 20:15:00,2010-02-28 12:38:00,https://www.flickr.com/photos/11545749@N06/439...,"[avott, gift, nin, nineinchnails, screening, t...",6,1,"[2010-01-29, Toiou, Avott]","[nin,nineinchnails,gift,screening,toiou,avott]",[Parc]
4,4394803554,11545749@N06,45.784000,4.874072,"lyon,nin,nineinchnails,gift,screening,toiou,avott",2010-01-28 Toiou Avott Lyon,10.0,20,28,1,...,2010.0,2010-01-28 20:10:00,2010-02-28 12:38:00,https://www.flickr.com/photos/11545749@N06/439...,"[avott, gift, lyon, nin, nineinchnails, screen...",7,1,"[2010-01-28, Toiou, Avott]","[lyon,nin,nineinchnails,gift,screening,toiou,a...",[Parc]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168096,45746643811,146709606@N06,45.729498,4.951550,"car,autocar,interurbain,ligne,scolaire,temsa,l...",TEMSA LD 12 SB Plus - DGC,32.0,15,17,10,...,2018.0,2018-10-17 15:32:00,2018-11-06 11:40:00,https://www.flickr.com/photos/146709606@N06/45...,"[autocar, car, dcg, dietrichcarebusgroup, inte...",11,3,"[TEMSA, Plus, DGC]","[car,autocar,interurbain,ligne,scolaire,temsa,...",[Japan]
168097,45746642111,146709606@N06,45.729498,4.951550,"car,autocar,interurbain,ligne,scolaire,temsa,l...",TEMSA LD 12 SB Plus - DGC,33.0,15,17,10,...,2018.0,2018-10-17 15:33:00,2018-11-06 11:40:00,https://www.flickr.com/photos/146709606@N06/45...,"[autocar, car, dcg, dietrichcarebusgroup, inte...",11,3,"[TEMSA, Plus, DGC]","[car,autocar,interurbain,ligne,scolaire,temsa,...",[Japan]
168098,45680287992,146709606@N06,45.729498,4.951550,"car,autocar,tourisme,interurbain,van,hool,vanh...",VAN HOOL TDX27 Astromega - Van Hool,52.0,13,17,10,...,2018.0,2018-10-17 13:52:00,2018-11-05 13:00:00,https://www.flickr.com/photos/146709606@N06/45...,"[2018, 27, astromega, autocar, autocarexpo, ca...",16,3,"[VAN, HOOL, TDX27, Astromega, Van, Hool]","[car,autocar,tourisme,interurbain,van,hool,van...",[Japan]
168099,44995017704,48633948@N08,45.771852,4.833115,"lyon,1714,4814we69,man,hess,nmt222,trolleybus,...",Lyon 1714 Rue Pouteau,20.0,14,18,10,...,2018.0,2017-10-18 14:20:00,2018-11-04 21:19:00,https://www.flickr.com/photos/48633948@N08/449...,"[1714, 4814we69, bus, buses, france, hess, lyo...",12,0,"[1714, Rue, Pouteau]","[lyon,1714,4814we69,man,hess,nmt222,trolleybus...",[France]]
